In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
sy.logger.remove()
import numpy as np
import pandas as pd
from syft.core.adp.entity import Entity
from syft.core.tensor.smpc.mpc_tensor import MPCTensor
import time
import uuid
from typing import Any
from typing import Dict
data = sy.Tensor(np.array([1,2,3],dtype=np.int32))

In [2]:
DOMAIN1_PORT = 8081
DOMAIN2_PORT = 8082
p1 =  sy.login(email="info@openmined.org",password="changethis",port=DOMAIN1_PORT)
p2 = sy.login(email="info@openmined.org",password="changethis",port=DOMAIN2_PORT)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into canada... done!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into italy... done!


In [3]:
ROOT_DIR = "/home/azureuser/PySyft"
def load_data(csv_file: str) -> pd.DataFrame:
    return pd.read_csv(f"{ROOT_DIR}/notebooks/trade_demo/datasets/{csv_file}")[0:10]

def get_user_details(unique_email: str) -> Dict[str, Any]:
    return {
        "name": "Sheldon Cooper",
        "email": unique_email,
        "password": "bazinga",
        "budget": 10,
    }

unique_email = f"{uuid.uuid4()}@caltech.edu"

# Canada
ca_root = sy.login(email="info@openmined.org", password="changethis", port=DOMAIN1_PORT)
ca_data = load_data(csv_file="ca - feb 2021.csv")

# NOTE: casting this tensor as np.int32 is REALLY IMPORTANT
canada_trade = (
    (np.array(list(ca_data["Trade Value (US$)"])) / 100000)[0:10]
).astype(np.int32)
trade_partners = ((list(ca_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(Entity(name=trade_partners[i]))

sampled_canada_dataset = sy.Tensor(canada_trade)
sampled_canada_dataset.public_shape = sampled_canada_dataset.shape
sampled_canada_dataset = sampled_canada_dataset.private(
    0, 3, entities=entities[0]
).tag("trade_flow")

# load dataset
ca_root.load_dataset(
    assets={"Canada Trade": sampled_canada_dataset},
    name="Canada Trade Data - First few rows",
    description=(
        "A collection of reports from Canada's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
)

assert len(ca_root.datasets) > 0



# Italy
it_root = sy.login(email="info@openmined.org", password="changethis", port=DOMAIN2_PORT)
it_data = load_data(csv_file="it - feb 2021.csv")
# NOTE: casting this tensor as np.int32 is REALLY IMPORTANT
data_batch = ((np.array(list(it_data["Trade Value (US$)"])) / 100000)[0:10]).astype(
    np.int32
)
trade_partners = ((list(it_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(Entity(name="Other Asia, nes"))

# Upload a private dataset to the Domain object, as the root owner
sampled_italy_dataset = sy.Tensor(data_batch)
sampled_italy_dataset.public_shape = sampled_italy_dataset.shape
sampled_italy_dataset = sampled_italy_dataset.private(
    0, 3, entities=entities[0]
).tag("trade_flow")

it_root.load_dataset(
    assets={"Italy Trade": sampled_italy_dataset},
    name="Italy Trade Data - First few rows",
    description=(
        "A collection of reports from iStat's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
)




Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into canada... done!


/tmp/ipykernel_2136698/1115671157.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{ROOT_DIR}/notebooks/trade_demo/datasets/{csv_file}")[0:10]


Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into italy... done!
Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!


/tmp/ipykernel_2136698/1115671157.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{ROOT_DIR}/notebooks/trade_demo/datasets/{csv_file}")[0:10]


In [13]:
x = p1.datasets[-1]["Canada Trade"]

In [14]:
y = p2.datasets[-1]["Italy Trade"]

In [12]:
sampled_canada_dataset.decode()

array([ 0.,  1., 14.,  0.,  0.,  0.,  0.,  3.,  0.,  0.], dtype=float32)

In [10]:
sampled_italy_dataset.decode()

array([ 0.,  1.,  3.,  0.,  1., 14.,  0.,  1.,  0.,  0.], dtype=float32)

In [16]:
t1 = x.get_copy()

In [17]:
t1

Tensor(child=SingleEntityPhiTensor(entity=Other Asia, nes, child=FixedPrecisionTensor(child=FixedPrecisionTensor(child=FixedPrecisionTensor(child=FixedPrecisionTensor(child=[0 0 0 0 0 0 0 0 0 0]))))))

In [ ]:
x = p1.datasets[0]["Canada Trade"]

In [ ]:
val = x.get_copy()

In [ ]:
ptr = val.send(p1)

In [ ]:
ptr

In [ ]:
t=x*y

In [ ]:
t1 = x.get_copy()

In [ ]:
t2 = y.get_copy()

In [ ]:
t = x+y

In [ ]:
i = t.child[0].get_copy()

In [ ]:
j = t.child[1].get_copy()

In [ ]:
n= x>y

In [ ]:
i

In [ ]:
public_result = t.publish(sigma=2)

In [ ]:
t.get()

In [ ]:
time.sleep(5)
sycure_result = public_result.get()
print("sycure_result", sycure_result)
print("after ca", ca.privacy_budget)
print("after it", it.privacy_budget)

assert len(sycure_result) == 10
assert sum(sycure_result) > -100
assert sum(sycure_result) < 100

assert ca.privacy_budget < 200
assert ca.privacy_budget > 10
assert it.privacy_budget < 200
assert it.privacy_budget > 10
assert ca.privacy_budget == it.privacy_budget